## Import required library

In [2]:
import os                                     # interact with the operating system
import shutil                                 # operating on file like copying, create and del directories and file
from PIL import Image                         # image processing 
from multiprocessing.pool import ThreadPool   # parallelizing the process
from tqdm import tqdm                         # for progess bar
import glob                                   # for searcing the file with specific file pattern or name

In [51]:
import torch
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [28]:
"""processed_root = "data"

os.makedirs(processed_root, exist_ok=True)
for class_dir in os.listdir(original_root):
    class_path = os.path.join(original_root, class_dir)
    if os.path.isdir(class_path):
        frames_path = os.path.join(class_path, "frames_sampled30")
        if os.path.exists(frames_path):
            processed_class_path = os.path.join(processed_root, class_dir)
            os.makedirs(processed_class_path, exist_ok=True)
            # move all the images to the processed_class_path
            for img_file in os.listdir(frames_path):
                src = os.path.join(frames_path, img_file)
                dest = os.path.join(processed_class_path, img_file)
                shutil.copy(src, dest)

print(f"Data preprocessing Completed")  """

'processed_root = "data"\n\nos.makedirs(processed_root, exist_ok=True)\nfor class_dir in os.listdir(original_root):\n    class_path = os.path.join(original_root, class_dir)\n    if os.path.isdir(class_path):\n        frames_path = os.path.join(class_path, "frames_sampled30")\n        if os.path.exists(frames_path):\n            processed_class_path = os.path.join(processed_root, class_dir)\n            os.makedirs(processed_class_path, exist_ok=True)\n            # move all the images to the processed_class_path\n            for img_file in os.listdir(frames_path):\n                src = os.path.join(frames_path, img_file)\n                dest = os.path.join(processed_class_path, img_file)\n                shutil.copy(src, dest)\n\nprint(f"Data preprocessing Completed")  '

In [31]:
# calculating the total number of directory and images in each directory
processed_root="/home/ritushwar/Nutritional-Assistance/data"
total_img = 0
imgs_per_dir = []  # for making the list of images in each directory
total_dirs = 0     # for calculating total no of directories
for dir in os.listdir(processed_root): 
    processed_class_path = os.path.join(processed_root, dir)
    if os.path.isdir(processed_class_path):
        total_dirs +=1
        img = 0            # for calculating total no of images in each directory
        for im in os.listdir(processed_class_path):
            img_path = os.path.join(processed_class_path, im)
            if os.path.isfile(img_path):
                img += 1
                total_img +=1
        imgs_per_dir.append(img)
        if img == 29:
            print(processed_class_path)
        img = 0

print("Done")

Done


In [34]:
indx = imgs_per_dir.index(28, 0, -1)
print(indx)

121


In [32]:
# finding the unique number of images
unq_img = set(imgs_per_dir)

In [35]:
print(f"Total no of directory: {total_dirs}")
print(f"Total no of images: {total_img}")
print(f"Unique set of images per directory: {unq_img}")

Total no of directory: 4571
Total no of images: 49647
Unique set of images per directory: {2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28}


## Resize the image

In [84]:
# using pillow library
img_path = "/home/ritushwar/Nutritional-Assistance/camera_A_frame_003.jpeg"
image = Image.open(img_path)
print(image.size)
image.show()

(1920, 1080)


In [85]:
# using resize method
new_image = image.resize((200,200))
new_image.show()

In [ ]:
# using thumbnail method
image.thumbnail((320,320))
image.show()

In [88]:
for dir in os.listdir(processed_root):
    processed_class_path = os.path.join(processed_root, dir)
    if os.path.isdir(processed_class_path):
        for img in os.listdir(processed_class_path):
            image_path = os.path.join(processed_class_path, img)
            if os.path.isfile(image_path):
                org_image = Image.open(image_path)
                image.thumbnail((320, 180))
                image.save(image_path)

print("All image are resized")

All image are resized


In [ ]:
#converting into tensor for iteration
data_path = processed_root
data_tensor = datasets.ImageFolder(
    root=data_path,
    transform=transforms.ToTensor(),
    target_transform=None
)

In [90]:
image, label = data_tensor[0]
print(image.shape)
print(label)

torch.Size([3, 180, 320])
0


In [92]:
classes = data_tensor.classes